# Laboratorium 6

##  Składanie i "aplikacja" funkcji: funkcje postaci: vs.(rozszerzone/monadyczne, Kleisli arrows)

In [ ]:
(<$<) :: (a -> b) -> a -> b 
(<$<) = ($) 
 
(>$>) :: a -> (a -> b) -> b 
x >$> f = f x 
infixl 0 >$> 

(\x -> 2 * x + 1) <$< 1 
1 >$> (\x -> 2 * x + 1) 

In [ ]:
(<.<) :: (b -> c) -> (a -> b) -> (a -> c) 
(<.<) = (.) 
 
(>.>) :: (a -> b) -> (b -> c) -> (a -> c) 
f >.> g = g . f 
infixl 9 >.> 


(+2) <.< (*3) <$< 5 
(+2) >.> (*3) <$< 5 
5 >$> (+2) <.< (*3) 
5 >$> (+2) >.> (*3) 
3 >$> (+2) >$> (*10) 

In [ ]:
tail [1,2] 
tail >.> tail $ [1,2] 

:t tail

safeTail :: [a] -> Maybe [a] 
safeTail []     = Nothing 
safeTail (x:xs) = Just xs 

safeTail [1,2] 
safeTail [] 
:t safeTail 
-- safeTail >.> safeTail >.> safeTail <$< [1,2] Cause we are not getting a in there
-- [1,2] >$> safeTail >.> safeTail >.> safeTail 

-- Nie da się, bo safeTail nie zwraca a, a zwraca Maybe a

In [ ]:
extractMaybe :: Maybe a -> a 
extractMaybe Nothing  = error "Nothing inside!" 
extractMaybe (Just x) = x 

[1,2] >$> safeTail >.> extractMaybe >.> safeTail 
[1,2] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail
[1] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail 

In [ ]:
insertMaybe :: a -> Maybe a 
insertMaybe = Just 
 
-- (>^$>) = extract (^) and apply ($) 
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b 
ma >^$> f = (extractMaybe ma) >$> f 
infixl 1 >^$> 

In [ ]:
insertMaybe [1,2] 
[1,2] >$> insertMaybe >.> extractMaybe 
-- [1,2] >^$> safeTail 
:t (>^$>) 
insertMaybe [1,2] >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail 

In [ ]:
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b 
Nothing  >^$> _ = Nothing 
(Just x) >^$> f = f x 
infixl 1 >^$> 


insertMaybe [1,2] >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail 
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail 

In [ ]:
f1 :: (Ord a, Num a) => a -> Maybe a 
f1 x = if x > 0 then Just (x + 1) else Nothing 
 
f2 :: (Eq a, Num a) => a -> Maybe a 
f2 x = if x /= 0 then Just (10 * x) else Nothing 
 
-- Kleisli composition 
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c) 
f >.>> g = \x -> g (extractMaybe (f x)) 

h = f1 >.>> f2 
:t h 
f1 3 
f2 3 
h 3 

In [ ]:
let f3 x = Just (1/x) 
let f4 x = Just (2 * x) 
:t f3 
:t f4 
let f34 = f3 >.>> f4 
:t f34 
f34 2 

## Zadania

In [ ]:
-- Zad 1
-- f >.>> g = \x -> g (extractMaybe (f x)) 
f >.>> g = \x -> (f x) >^$> g 
let f3 x = Just (1/x) 
let f4 x = Just (2 * x) 
:t f3 
:t f4 
let f34 = f3 >.>> f4 
:t f34 
f34 2 

## Przykłady Monad: Maybe

In [ ]:
:i Maybe 
:i Monad 
return 1 :: Maybe Int 

In [ ]:
import Control.Monad 
:t (>=>) 

In [ ]:
insertMaybe [1,2] 
return (Just [1,2]) 
return [1,2] 
return [1,2] :: Maybe [Int] 
insertMaybe [1,2] >^$> safeTail 
return [1,2] >>= safeTail 
(return [1,2] >>= safeTail) == (insertMaybe [1,2] >^$> safeTail) 
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail 
return [1,2] >>= safeTail >>= safeTail >>= safeTail >>= safeTail 
return [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail 
insertMaybe [1,2] >>= safeTail >^$> safeTail >>= safeTail >^$> safeTail
return [1,2] >^$> safeTail >>= safeTail >^$> safeTail >>= safeTail 

In [ ]:
doSafeTail3x :: [a] -> Maybe [a]
doSafeTail3x xs = do 
  t1 <- safeTail xs 
  t2 <- safeTail t1 
  t3 <- safeTail t2 
  return t3 
 
safeTail3x :: [a] -> Maybe [a] 
safeTail3x xs = 
  safeTail xs >>= \t1 -> 
    safeTail t1 >>= \t2 -> 
      safeTail t2 >>= \t3 -> 
        return t3 
safeTail3x' :: [a] -> Maybe [a] 
safeTail3x' xs = return xs >>= safeTail >>= safeTail >>= safeTail


In [ ]:
doSafeTail3x [1..5] 
safeTail3x [1..5] 
safeTail3x' [1..5] 
doSafeTail3x [1] 
safeTail3x [1] 
safeTail3x' [1] 

In [ ]:
f5 :: Int -> Int -> Int -> Int 
f5 x y z = 1000 `div` x + 100 `div` y + 10 `div` z 

f5 1 2 3 
f5 1 2 0 
f5 1 0 3 
f5 0 2 3 

In [ ]:
safeDiv :: Int -> Int -> Maybe Int 
safeDiv x y | y /= 0    = Just $ x `div` y 
            | otherwise = Nothing 
 
safeF5 :: Int -> Int -> Int -> Maybe Int 
safeF5 x y z = 
  case (safeDiv 1000 x) of 
    Nothing -> Nothing 
    Just (iOverX) -> 
      case (safeDiv 100 y) of 
        Nothing -> Nothing 
        Just (iOverY) -> 
          case (safeDiv 10 z) of
            Nothing -> Nothing 
            Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ 

In [ ]:
safeF5 1 2 3 
safeF5 1 2 0 
safeF5 1 0 3 
safeF5 0 2 3 


safeF5' :: Int -> Int -> Int -> Maybe Int 
safeF5' x y z = do 
  iOverX <- safeDiv 1000 x 
  iOverY <- safeDiv 100 y 
  iOverZ <- safeDiv 10 z 
  return $ iOverX + iOverY + iOverZ 

## Zadania

In [ ]:
-- Zad 1 (Dla monady Maybe ) zdefiniować (>=>) 
-- przy pomocy >>= ; czy można tę definicję
-- uogólnić, aby była prawdziwa dla dowolnej monady?
-- import Control.Monad 
-- (>>=) :: Maybe a -> (a -> Maybe b) -> Maybe b
-- (>>=) m g = case m of
--                Nothing -> Nothing
--                Just x  -> g x
               
(>=>) :: Monad m => (a -> m b) -> (b -> m c) -> a -> m c
f >=> g = \x -> f x >>= g
-- Tak, jest prawdziwa dla dowolnej monady

In [ ]:
-- Zad 2
:i Maybe

In [ ]:
-- Zad 3
joinMaybe :: Monad m => m (m a) -> m a
joinMaybe x = x >>= id

## Przykłady monad: []

In [ ]:
:i [] 
[1,2,3] :: [Int] 
[1,2,3] :: [] Int 

In [ ]:
return 1 :: [] Int 
return 3 >>= (\x -> [1..x]) 
[1,2] >>= (\x -> [-x,x]) 
[1,2,3] >>= (\x -> [-x..x]) 
[1,2] >>= (\x -> [-x,x]) >>= (\y -> [-y,y]) 
[1,2] >>= \x -> [-x,x] >>= \y -> [-y,y] 
[1,2] >>= \x -> [-x,x] >>= \y -> return (x,y) 

In [ ]:
xs1 :: [(Int,Int,Int)] 
xs1 = [ (x,y,z) | let xs = [1,2], x <- xs, y <- xs, z <- xs ] 
 
doXs1 :: [(Int,Int,Int)] 
doXs1 = do 
  let xs = [1,2] 
  x <- xs 
  y <- xs 
  z <- xs 
  return (x,y,z) 
xs1 
doXs1 
xs1 == doXs1 

In [ ]:
import Control.Monad 
xs2 :: [(Int,Int,Int)] 
xs2 = [ (x,y,z) | let xs = [1..3], x <- xs, y <- xs, z <- xs, x > y && y > z ] 
 
doXs2 :: [(Int,Int,Int)] 
doXs2 = do 
  let xs = [1..3] 
  x <- xs 
  y <- xs 
  z <- xs 
  guard $ x > y && y > z 
  return (x,y,z) 
 
doXs2' :: [(Int,Int,Int)] 
doXs2' = do 
  let xs = [1..3] 
  x <- xs 
  y <- xs 
  z <- xs 
  if x > y && y > z 
    then return (x,y,z) 
    else [] 
    

:t guard 
[1..10] >>= \x -> guard (even x) >> return x 
[1..10] >>= \x -> guard (even x) >>= \y -> return (x,y) 
[1..10] >>= \x -> return x
[1..10] >>= return 

## Zadania

In [ ]:
join :: Monad m => m (m a) -> m a
join x = x >>= id